# 1. 강의영상 

# 2. Imports 

In [394]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn.linear_model 
import sklearn.tree
import sklearn.model_selection

# 3. Data 

In [353]:
import pandas as pd
import numpy as np

# 데이터 크기 설정
n = 10000

# 무작위로 복용 여부와 운동량을 생성
Supplement = np.random.choice([True, False], n)
Exercise = np.random.choice([False, True], n)

# 체중 감량 여부를 계산 (이 부분은 수정하지 않습니다)

# 연속형 변수 생성
Weight_Loss = np.where(
    (~Supplement & (~Exercise)),
    np.random.normal(loc=0, scale=1, size=n),  # 낮은 운동량에서 체중 감량
    np.where(
        (Supplement & (Exercise)),
        np.random.normal(loc=15.00, scale=1, size=n),  # 높은 운동량에서 체중 감량
        np.where(
            (~Supplement & (Exercise)),
            np.random.normal(loc=5.00, scale=1, size=n),  # 낮은 운동량에서 체중 증가
            np.random.normal(loc=0.5, scale=1, size=n)   # 높은 운동량에서 체중 증가
        )
    )
)

# 데이터프레임 생성
df = pd.DataFrame({
    'Supplement': Supplement,
    'Exercise': Exercise,
    'Weight_Loss': Weight_Loss
})

# 데이터 확인
df.head()

,Supplement,Exercise,Weight_Loss
0,True,False,-0.366171
1,True,False,0.537226
2,True,False,-0.628373
3,False,True,5.136422
4,True,True,15.533363


In [419]:
#df.to_csv('weightloss.csv',index=False)

In [354]:
# import pandas as pd
# import numpy as np

# # 데이터 크기 설정
# n = 10000

# # 무작위로 복용 여부와 운동량을 생성
# Supplement = np.random.choice([True, False], n)
# Exercise = np.random.choice(['Low', 'High'], n)

# # 체중 감량 여부를 계산
# Weight_Loss = np.where(
#     (~Supplement & (Exercise == 'Low')),
#     np.random.binomial(1, 0.05, n),
#     np.where(
#         (Supplement & (Exercise == 'High')),
#         np.random.binomial(1, 0.95, n),
#         np.where(
#             (~Supplement & (Exercise == 'High')),
#             np.random.binomial(1, 0.55, n),
#             np.random.binomial(1, 0.25, n)
#         )
#     )
# )

# # 데이터프레임 생성
# df = pd.DataFrame({
#     'Supplement': Supplement,
#     'Exercise': Exercise,
#     'Weight_Loss': Weight_Loss
# })

# # 데이터 확인
# print(df.head())
df = pd.read_csv('https://raw.githubusercontent.com/guebin/MP2023/main/posts/weightloss.csv')

In [355]:
df.pivot_table(index='Supplement',columns='Exercise',values='Weight_Loss')

Exercise,False,True
Supplement,,
False,-0.011756,4.975891
True,0.492372,14.987283


`-` 운동과 체중감량보조제를 병행하면 시너지가 나는 것 같음 

# 4. 분석1

`-` 분석1: 모형을 아래와 같이 본다. 

- ${\bf X}$: `Supplement`, `Exercise`
- ${\bf y}$: `Weight_Loss`

In [414]:
# step1
X = pd.get_dummies(df[['Supplement','Exercise']])
y = df['Weight_Loss']
# step2 
predictr = sklearn.linear_model.LinearRegression()
# step3
predictr.fit(X,y)
# step4 
yhat = predictr.predict(X)

In [415]:
df.assign(yhat = yhat)

,Supplement,Exercise,Weight_Loss,yhat
0,False,False,-0.877103,-2.373106
1,True,False,1.604542,2.845934
2,True,True,13.824148,12.593598
3,True,True,13.004505,12.593598
4,True,True,13.701128,12.593598
...,...,...,...,...
9995,True,False,1.558841,2.845934
9996,False,False,-0.217816,-2.373106
9997,False,True,4.072701,7.374557
9998,True,False,-0.253796,2.845934


In [416]:
df.assign(yhat = yhat).pivot_table(index='Supplement',columns='Exercise',values='yhat')

Exercise,False,True
Supplement,,
False,-2.373106,7.374557
True,2.845934,12.593598


In [417]:
predictr.score(X,y)

0.8208414124769221

# 5. 분석2

`-` 모형을 아래와 같이 본다. 

- ${\bf X}$: `Supplement`, `Exercise`, `Supplement` $\times$ `Exercise`
- ${\bf y}$: `Weight_Loss`

> Note: 기본적인 운동의 효과 및 보조제의 효과는 각각 `Supplement`, `Exercise` 로 적합하고 운동과 보조제의 시너지는 `Supplement`$\times$`Exercise` 로 적합한다. 

In [408]:
# step1 
X = pd.get_dummies(df.assign(Supp_Exer = df.Supplement * df.Exercise).drop(['Weight_Loss'],axis=1))
y = df['Weight_Loss']
# step2 
predictr = sklearn.linear_model.LinearRegression()
# step3 
predictr.fit(X,y)
# step4 -- pass 
#---# 
yhat = predictr.predict(X)

In [409]:
df.assign(yhat = yhat)

,Supplement,Exercise,Weight_Loss,yhat
0,False,False,-0.877103,0.021673
1,True,False,1.604542,0.497573
2,True,True,13.824148,14.966363
3,True,True,13.004505,14.966363
4,True,True,13.701128,14.966363
...,...,...,...,...
9995,True,False,1.558841,0.497573
9996,False,False,-0.217816,0.021673
9997,False,True,4.072701,4.991314
9998,True,False,-0.253796,0.497573


In [410]:
df.assign(yhat = yhat).pivot_table(index='Supplement',columns='Exercise',values='yhat')

Exercise,False,True
Supplement,,
False,0.021673,4.991314
True,0.497573,14.966363


In [412]:
predictr.score(X,y)

0.9727754257714795